# HR/LR Synthetic Image Generation  



**Notebook**: Creates paired HR (256px) and LR (64px) patches with:  
- Blur degradation  
- Poisson + sensor noise  
- Bicubic/bilinear downsampling  
<br>

#1 Import dependencies

In [ ]:
from PIL import Image, ImageFilter
from google.colab import drive, files
import matplotlib.pyplot as plt
import os
import cv2
import numpy as np
import io
import zipfile

#2 Upload image

In [ ]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


#3 Create patches of 256 by 256 pixels

In [ ]:
# Create output folder
output_dir = "patches_256"
os.makedirs(output_dir, exist_ok=True)

In [ ]:
def extract_patches(image, width, height, output_dir, prefix):
    """Extract square patches from an image and save as PNG files.

    Args:
        image: PIL.Image - Source image to split into patches
        width: int - Image width in pixels (must match actual image width)
        height: int - Image height in pixels (must match actual image height)
        output_dir: str - Directory to save patches (e.g. '/content/patches')
        prefix: str - Prefix for filenames (e.g. 'Nepal')

    Output:
        Saves files: {output_dir}/{prefix}_patch_N.png (N starts at 0)
        Prints total number of patches saved

    How it works:
        - Slides a 256x256 window across the image with no overlap
        - Saves each patch as separate PNG
        - Uses global 'prefix' for filenames (e.g. 'Nepal_patch_0.png')
    """
    patch_size=256
    patch_id = 0

    for top in range(0, height - patch_size + 1, patch_size):
        for left in range(0, width - patch_size + 1, patch_size):
            image.crop((left, top, left + patch_size, top + patch_size)) \
               .save(f"{output_dir}/{prefix}_patch_{patch_id}.png")
            patch_id += 1

    print(f"Saved {patch_id} patches (using pre-calculated size {width}x{height})")

In [ ]:
input_dir = "/content/drive/MyDrive/Final_project/OriginalImages/SatelliteImages"
output_dir = "/content/patches_256"
os.makedirs(output_dir, exist_ok=True)

# Get all PNG files in the directory
png_files = [f for f in os.listdir(input_dir) if f.lower().endswith('.png')]

# Process each image
for png_file in png_files:
    img_path = os.path.join(input_dir, png_file)
    print(f"\nProcessing image: {img_path}")
    prefix = os.path.splitext(png_file)[0]
    # Load image and print dimensions for verification
    image = Image.open(img_path)
    width, height = image.size
    print(f"Image size: {width} x {height}")

    # Call patch extraction function
    extract_patches(image, width, height, output_dir, prefix)

    image.close()


Processing image: /content/drive/MyDrive/Final_project/OriginalImages/SatelliteImages/buildings.png
Image size: 2549 x 2500
Saved 81 patches (using pre-calculated size 2549x2500)


#4 Create HR images

##4.1 Check lapalcian pyramid

In [ ]:
def is_high_quality(patch):
    """Determine if an image patch has sufficient texture and sharpness.

    Args:
        patch: PIL.Image - Input image patch to evaluate

    Returns:
        bool: True if patch passes both quality checks:
            - Standard deviation >= 25 (contrast/texture)
            - Average Laplacian variance across pyramid levels >= 80

    How it works:
        1. Converts patch to numpy array
        2. Rejects if pixel intensity std dev < 25 (flat regions)
        3. Rejects if average Laplacian pyramid variance < 80 (blurry/soft edges across scales)
    """
    patch_np = np.array(patch)

    # Standard deviation check (contrast/texture)
    sd = np.std(patch_np)
    if sd < 25:
        return False

    # Convert to grayscale
    gray = cv2.cvtColor(patch_np, cv2.COLOR_RGB2GRAY)

    # Build Laplacian pyramid with 3 levels to check
    max_levels = 3
    pyramid_vars = []

    current = gray.copy()
    for _ in range(max_levels):
        laplacian = cv2.Laplacian(current, cv2.CV_64F)
        pyramid_vars.append(np.var(laplacian))
        current = cv2.pyrDown(current)  # downsample for next level

    # Take average sharpness measure across levels
    avg_var = np.mean(pyramid_vars)

    if avg_var < 80:
        return False

    return True


In [ ]:
def rename_low_quality_patches(output_dir):
    """Renames low-quality image patches by adding 'low_quality_' prefix.

    Args:
        output_dir: Directory containing the patch PNG files to check

    Output:
        Renames PNG files flagged as low qaulity
        Prints renaming actions to console

    How it works:
        1. Finds all files matching '{prefix}_patch_*.png'
        2. Checks each with is_high_quality()
        3. Renames and logs low quality patches
    """
    for filename in os.listdir(output_dir):
        if filename.endswith(".png"):
            filepath = os.path.join(output_dir, filename)
            patch = Image.open(filepath)

            if not is_high_quality(patch):
                new_name = "low_quality_" + filename
                new_path = os.path.join(output_dir, new_name)
                os.rename(filepath, new_path)
                print(f"Renamed {filename} to {new_name}")

In [ ]:
rename_low_quality_patches(output_dir)

##4.2 Sharpen HR images

In [ ]:
sharpened_dir = 'hr_patches_256'
os.makedirs(sharpened_dir, exist_ok=True)

In [ ]:
# =============================================================================
# PATCH SHARPENING CONFIGURATION
# =============================================================================
radius = 2      # Smaller = finer details (1.0-2.0)
percent = 50    # Strength (50-100)
threshold = 5   # Only sharpen areas with contrast above this (0-10)

# =============================================================================
# SHARPENING PROCESS
# =============================================================================
"""
Process:
    1. Scans output_dir for patch images
    2. Skips already sharpened/low-quality files
    3. Applies unsharp mask with current settings
    4. Saves sharpened versions to sharpened_dir

Output:
    - Creates sharp_*.png copies in sharpened_dir
    - Prints success count and error messages
    - Preserves original files
"""
processed_count = 0

for patch_path in os.listdir(output_dir):

    full_path = os.path.join(output_dir, patch_path)
    # Skip directories like .ipynb_checkpoints
    if os.path.isdir(full_path):
        continue

    # Skip files that are already sharpened or marked as low quality
    if patch_path.startswith(('sharp_', 'low_quality_')):
        continue

    try:
        with Image.open(os.path.join(output_dir, patch_path)) as img:
            # Apply sharpening
            sharp = img.filter(
                ImageFilter.UnsharpMask(
                    radius=radius,
                    percent=percent,
                    threshold=threshold
                )
            )

            # Save with sharp_ prefix
            sharp.save(os.path.join(sharpened_dir, patch_path))
            processed_count += 1

    except Exception as e:
        print(f"Error processing {patch_path}")

print(f"Sharpened {processed_count} images. Saved to {sharpened_dir}/")

Sharpened 81 images. Saved to hr_patches_256/


#5 Create LR images

In [ ]:
src_dir = "/content/drive/MyDrive/Final_project/OriginalImages/SatelliteImages"

##5.1 Image blur

In [ ]:

# =============================================================================
# DEGRADATION PARAMETER OPTIONS
# =============================================================================
"""
- motion_length: Blur distance in pixels (3-7)
- motion_angle: Blur direction in degrees (45 degrees increments)
- defocus_radius: Out-of-focus blur strength (3-5)
- gaussian_sigma: General blur intensity (0.5-1.2)
- resampling_type: Downscaling method (bicubic or bilinear)
"""
param_ranges = {
    'motion_length': [3, 7],
    'motion_angle': [45, 90, 135, 180],
    'defocus_radius': [3, 5],
    'gaussian_sigma': [0.5, 1.2],
    'resampling_type': ['bicubic', 'bilinear']
}

def get_random_degradation():
    """Generates random image degradation parameters.

    Returns:
        dict: Randomly selected values for all parameters in param_ranges
    """
    return {key: np.random.choice(values) for key, values in param_ranges.items()}

# Random param values
random_params = get_random_degradation()
print("Random degradation parameters:", random_params)

Random degradation parameters: {'motion_length': np.int64(3), 'motion_angle': np.int64(90), 'defocus_radius': np.int64(5), 'gaussian_sigma': np.float64(0.5), 'resampling_type': np.str_('bicubic')}


In [ ]:
# Define blur degradation functions
def apply_motion_blur(img, L, F):
    """Applies directional motion blur to an image.

    Args:
        img: Input image (numpy array)
        L: Length of motion blur
        F: Angle of motion in degrees

    Returns:
        np.ndarray: Image with motion blur applied

    How it works:
        1. Creates horizontal motion kernel
        2. Rotates kernel to specified angle
        3. Applies using 2D convolution
    """
    kernel = np.zeros((L, L))
    center = L // 2
    kernel[center, :] = np.ones(L) / L
    M = cv2.getRotationMatrix2D((float(center), float(center)), F, 1)
    kernel = cv2.warpAffine(kernel, M, (L, L))
    return cv2.filter2D(img, -1, kernel)

def apply_defocus_blur(img, r):
    """Applies circular defocus blur to simulate out-of-focus effect.

    Args:
        img: Input image (numpy array)
        r: Radius of blur circle (pixels)

    Returns:
        np.ndarray: Image with defocus blur

    How it works:
        1. Creates circular kernel
        2. Normalizes kernel values
        3. Applies using 2D convolution
    """
    kernel = np.zeros((2*r+1, 2*r+1))
    cv2.circle(kernel, (r, r), r, 1, -1)
    kernel /= kernel.sum()
    return cv2.filter2D(img, -1, kernel)

def apply_gaussian_blur(img, sigma):
    """Applies Gaussian blur for general smoothing.

    Args:
        img: Input image (numpy array)
        sigma: Standard deviation of Gaussian kernel

    Returns:
        np.ndarray: Blurred image

    How it works:
        Uses fixed 25x25 kernel with specified sigma
        (Larger sigma = more blur)
    """
    kernel_size = 25
    return cv2.GaussianBlur(img, (kernel_size, kernel_size), sigmaX=sigma)


In [ ]:
# Applies degradations using random params generated previously
def process_image_cumulative(img):
    """Applies multiple degradations to an image using predefined random parameters.

    Args:
        img (np.ndarray): Input image loaded via OpenCV in BGR format.

    Returns:
        np.ndarray: Degraded image with combined effects

    How it works:
        1. Applies motion blur using random length/angle
        2. Applies defocus blur using random radius
        3. Applies Gaussian blur using random sigma
        4. Prints applied parameters for verification
    """
    degraded = img.copy()

    #degraded = apply_motion_blur(degraded,
                       #        random_params['motion_length'],
                       #        random_params['motion_angle'])


    degraded = apply_defocus_blur(degraded,
                                random_params['defocus_radius'])
    return degraded
'''
    degraded = apply_gaussian_blur(degraded,
                                 random_params['gaussian_sigma'])

    print(f"Applied from random_params: L={random_params['motion_length']}, "
          f"F={random_params['motion_angle']}°, "
          f"r={random_params['defocus_radius']}, "
          f"σ={random_params['gaussian_sigma']:.1f}")
'''


'\n    degraded = apply_gaussian_blur(degraded,\n                                 random_params[\'gaussian_sigma\'])\n\n    print(f"Applied from random_params: L={random_params[\'motion_length\']}, "\n          f"F={random_params[\'motion_angle\']}°, "\n          f"r={random_params[\'defocus_radius\']}, "\n          f"σ={random_params[\'gaussian_sigma\']:.1f}")\n'

In [ ]:
src_dir

'/content/drive/MyDrive/Final_project/OriginalImages/SatelliteImages'

In [ ]:
# =============================================================================
# IMAGE BLUR PROCESSING EXECUTION
# =============================================================================
"""
Process:
    1. Loads image from specified path using OpenCV (BGR format)
    2. Applies cumulative degradations using process_image_cumulative()
    3. Stores result in blurred_img variable

Note: Uses random_params generated earlier for degradation settings"""

dst_dir = "/content/blurred_images"
os.makedirs(dst_dir, exist_ok=True)

for img_name in os.listdir(src_dir):
    if img_name.lower().endswith(('.png')):
        src = os.path.join(src_dir, img_name)
        dst = os.path.join(dst_dir, img_name)

        img = cv2.imread(src)
        if img is not None:
            cv2.imwrite(dst, process_image_cumulative(img))
            print(f"Processed {img_name}")

Processed buildings.png


In [ ]:
"""
plt.imshow(cv2.cvtColor(blurred_img, cv2.COLOR_BGR2RGB))
plt.axis('off')
plt.show()
"""

"\nplt.imshow(cv2.cvtColor(blurred_img, cv2.COLOR_BGR2RGB))\nplt.axis('off')\nplt.show()\n"

##5.2 Noise Addition

###5.2.1 Add poisson noise - photon noise

In [ ]:
def add_poisson_noise(image_array, scale=1.3):
    """Adds Poisson (shot) noise to an image, preserving brightness distribution.

    Args:
        image_array: Input image (uint8 [0,255] or float [0,1])
        scale: Noise intensity multiplier (higher = more noise)

    Returns:
        np.ndarray: Noisy image (same datatype as input)

    How it works:
        1. Normalises image to [0,1] float if needed
        2. Scales intensity to control noise level
        3. Applies Poisson noise (photon counting statistics)
        4. Restores original range and dtype
    """
    if image_array.dtype == np.uint8:
        img_float = image_array.astype(np.float32) / 255.0
    else:
        img_float = image_array.copy()

    scaled_img = img_float * scale

    noisy_float = np.random.poisson(scaled_img * 255) / 255.0
    noisy_float = noisy_float / scale

    if image_array.dtype == np.uint8:
        return np.clip(noisy_float * 255, 0, 255).astype(np.uint8)
    else:
        return np.clip(noisy_float, 0, 1)

In [ ]:
# 1 Define paths
noisy_dir = "/content/noisy_images"
os.makedirs(noisy_dir, exist_ok=True)

# 2 Process each PNG
for img_name in os.listdir(dst_dir):
    if img_name.lower().endswith('.png'):
        # Load image
        img_path = os.path.join(dst_dir, img_name)
        blurred_img = cv2.imread(img_path)

        if blurred_img is not None:
            # Add Poisson noise
            noisy_img = add_poisson_noise(blurred_img, scale=1.0)

            # Save with same name to output directory
            cv2.imwrite(os.path.join(noisy_dir, img_name), noisy_img)
            print(f"Processed {img_name}")

Processed buildings.png


In [ ]:
'''
plt.imshow(cv2.cvtColor(noisy_img, cv2.COLOR_BGR2RGB))
plt.axis('off')
plt.show()
'''

"\nplt.imshow(cv2.cvtColor(noisy_img, cv2.COLOR_BGR2RGB))\nplt.axis('off')\nplt.show()\n"

###5.2.2 Add intrumental noise - thermal and quantisation

In [ ]:
def add_instrumental_noise(noisy_img, thermal_std=2.5, quantize=True):
    """Adds realistic sensor noise to an image (thermal + quantization noise).

    Args:
        noisy_img: Input image array (uint8 or float)
        thermal_std: Standard deviation of thermal noise (in 8-bit units)
        quantize: Whether to add quantization noise (for digital sensors)

    Returns:
        np.ndarray: Image with combined noise (uint8)

    How it works:
        1. Converts image to float32 for processing
        2. Adds Gaussian thermal noise
        3. Optionally adds uniform quantization noise (+- 0.5)
        4. Clips to valid range and returns as uint8
    """
    img_float = noisy_img.astype(np.float32)

    img_float += np.random.normal(0, thermal_std, size=noisy_img.shape)

    if quantize:
        img_float += np.random.uniform(-0.5, 0.5, size=noisy_img.shape)

    final_img = np.clip(img_float, 0, 255).astype(np.uint8)

    return final_img

In [ ]:
phy_noisy_dir = "/content/physically_noisy_image"
os.makedirs(phy_noisy_dir, exist_ok=True)


for img_name in os.listdir(noisy_dir):
    if img_name.endswith('.png'):
        img = cv2.imread(f"{noisy_dir}/{img_name}")
        if img is not None:
            noisy = add_poisson_noise(img, 1.0)
            final = add_instrumental_noise(noisy)
            cv2.imwrite(f"{phy_noisy_dir}/{img_name}", final)
            print(f"Processed {img_name}")

Processed buildings.png


In [ ]:
'''
plt.imshow(cv2.cvtColor(physically_noisy_img, cv2.COLOR_BGR2RGB))
plt.axis('off')
plt.show()
'''

"\nplt.imshow(cv2.cvtColor(physically_noisy_img, cv2.COLOR_BGR2RGB))\nplt.axis('off')\nplt.show()\n"

##5.3 create 256 by 256 patches

In [ ]:
# =============================================================================
# LOW-RESOLUTION PATCH GENERATION
# =============================================================================
"""
Process:
    1. Creates output directory lr_patches_256 if not exist
    2. Converts noisy BGR image to RGB format
    3. Converts numpy array to PIL Image format
    4. Extracts 256x256 patches using extract_patches()

Output:
    - Saves patches to lr_patches_256/
    - Filename format: {prefix}_patch_<id>.png
    - Prints total patch count when complete
"""
lr_output_dir = "lr_patches_256"
os.makedirs(lr_output_dir, exist_ok=True)

# Process each noisy image
for img_name in os.listdir(phy_noisy_dir):
    if img_name.endswith('.png'):
        # Load image
        img_path = os.path.join(phy_noisy_dir, img_name)
        physically_noisy_img = cv2.imread(img_path)

        if physically_noisy_img is not None:
            # Convert to PIL format
            physically_noisy_rgb = cv2.cvtColor(physically_noisy_img, cv2.COLOR_BGR2RGB)
            physically_noisy_pil = Image.fromarray(physically_noisy_rgb)

            # Get image dimensions
            width, height = physically_noisy_pil.size

            # Generate prefix from filename
            prefix = os.path.splitext(img_name)[0]

            # Extract and save patches
            extract_patches(physically_noisy_pil, width, height, lr_output_dir, prefix)
            print(f"Processed patches for: {img_name}")

Saved 81 patches (using pre-calculated size 2549x2500)
Processed patches for: buildings.png


In [ ]:
# =============================================================================
# PATCH SYNCHRONIZATION (HR-LR PAIR CLEANUP)
# =============================================================================
"""
Process:
    1. Identifies all HR patches in sharpened_dir
    2. Identifies all LR patches in lr_patches_256
    3. Finds LR patches without matching HR patches
    4. Deletes LR patches with no matching HR image

Output:
    - Removes inconsistent LR patches
    - Prints names of deleted patches
    - Reports total deletion count
"""

hr_patches = set(os.listdir(sharpened_dir))
lr_patches = set(os.listdir(lr_output_dir))

to_delete = lr_patches - hr_patches

for patch_name in to_delete:
    os.remove(os.path.join(lr_output_dir, patch_name))
    print(f"Deleted LR patch: {patch_name}")

print(f"Deleted {len(to_delete)} LR patches without HR counterparts")

Deleted 0 LR patches without HR counterparts


##5.4 Down sampling

In [ ]:
def apply_downsampling(noisy_img, scale_factor=4):
    """Downsamples an image while maintaining HR-LR correspondence.

      Args:
      noisy_img: Noisy input image (BGR format)
      scale_factor: Scaling ratio

      Returns:
      np.ndarray: Downsampled image in RGB format

      How it works:
      1. Calculates new dimensions (original // scale_factor)
      2. Applies either bicubic or bilinear resampling
      (based on random_params['resampling_type' ])
      3. Converts output to RGB color space
    """
    h, w = noisy_img.shape[:2]
    new_w, new_h = w // scale_factor, h // scale_factor

    # Use the resampling type from random_params
    if random_params['resampling_type'] == 'bicubic':
        lr_img = cv2.resize(noisy_img, (new_w, new_h), interpolation=cv2.INTER_CUBIC)
    else:
        lr_img = cv2.resize(noisy_img, (new_w, new_h), interpolation=cv2.INTER_LINEAR)

    return cv2.cvtColor(lr_img, cv2.COLOR_BGR2RGB)


# Path to newlr dir
output_dir = "lr_patches_64"
os.makedirs(output_dir, exist_ok=True)

# Process each patch
for patch_name in os.listdir(lr_output_dir):
    if patch_name.endswith('.png'):
        patch_path = os.path.join(lr_output_dir, patch_name)

        img = cv2.imread(patch_path)
        if img is not None:
            downsampled = apply_downsampling(img)
            save_path = os.path.join(output_dir, patch_name)
            cv2.imwrite(save_path, cv2.cvtColor(downsampled, cv2.COLOR_RGB2BGR))
            # print(f"Downsampled: {patch_name}")

print(f"All patches downsampled and saved in '{output_dir}'")


All patches downsampled and saved in 'lr_patches_64'


#6 Download HR and LR images

In [ ]:
# =============================================================================
# PATCH ARCHIVING & DOWNLOAD
# =============================================================================
"""
Final Output:
    1. HR Patches (256px):
       - Source: {sharpened_dir}/
       - Archive: hr_patches_256.zip
    2. LR Patches (64px):
       - Source: lr_patches_64/
       - Archive: lr_patches_64.zip

Process:
    1. Creates ZIP archives of both patch sehts
    2. Triggers download via Colab's files.download()
    3. Preserves directory structure in archives

Note:
  Appends new png images to existing hr_patches_256/ and lr_patches_64/
  For downloading inference images, comment lines for hr
"""
!zip -r hr_patches_256.zip {sharpened_dir}
files.download('hr_patches_256.zip')

!zip -r lr_patches_64.zip {output_dir}/
files.download('lr_patches_64.zip')

  adding: hr_patches_256/ (stored 0%)
  adding: hr_patches_256/buildings_patch_68.png (deflated 1%)
  adding: hr_patches_256/buildings_patch_47.png (deflated 1%)
  adding: hr_patches_256/buildings_patch_2.png (deflated 0%)
  adding: hr_patches_256/buildings_patch_66.png (deflated 0%)
  adding: hr_patches_256/buildings_patch_45.png (deflated 1%)
  adding: hr_patches_256/buildings_patch_42.png (deflated 0%)
  adding: hr_patches_256/buildings_patch_40.png (deflated 1%)
  adding: hr_patches_256/buildings_patch_64.png (deflated 0%)
  adding: hr_patches_256/buildings_patch_1.png (deflated 0%)
  adding: hr_patches_256/buildings_patch_28.png (deflated 0%)
  adding: hr_patches_256/buildings_patch_19.png (deflated 1%)
  adding: hr_patches_256/buildings_patch_75.png (deflated 0%)
  adding: hr_patches_256/buildings_patch_18.png (deflated 1%)
  adding: hr_patches_256/buildings_patch_37.png (deflated 0%)
  adding: hr_patches_256/buildings_patch_15.png (deflated 1%)
  adding: hr_patches_256/buildings

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

  adding: lr_patches_64/ (stored 0%)
  adding: lr_patches_64/buildings_patch_68.png (stored 0%)
  adding: lr_patches_64/buildings_patch_47.png (stored 0%)
  adding: lr_patches_64/buildings_patch_2.png (stored 0%)
  adding: lr_patches_64/buildings_patch_66.png (stored 0%)
  adding: lr_patches_64/buildings_patch_45.png (stored 0%)
  adding: lr_patches_64/buildings_patch_42.png (stored 0%)
  adding: lr_patches_64/buildings_patch_40.png (stored 0%)
  adding: lr_patches_64/buildings_patch_64.png (stored 0%)
  adding: lr_patches_64/buildings_patch_1.png (stored 0%)
  adding: lr_patches_64/buildings_patch_28.png (stored 0%)
  adding: lr_patches_64/buildings_patch_19.png (stored 0%)
  adding: lr_patches_64/buildings_patch_75.png (stored 0%)
  adding: lr_patches_64/buildings_patch_18.png (stored 0%)
  adding: lr_patches_64/buildings_patch_37.png (stored 0%)
  adding: lr_patches_64/buildings_patch_15.png (stored 0%)
  adding: lr_patches_64/buildings_patch_39.png (stored 0%)
  adding: lr_patches_

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>